# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
Classification, as the output is discrete, finite and known beforehand on the training set.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [ ]:
%matplotlib inline

# Import libraries
from __future__ import division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import time
from sklearn.metrics import f1_score
from IPython.display import display

# Read student data
student_data = pd.read_csv("student-data.csv")
display(student_data.head(5))

print "Student data read successfully!"

### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [ ]:
# Calculate number of students
n_students = len(student_data)

# Calculate number of features
n_features = len(student_data.columns) - 1  # Exclude the target label

# Calculate passing students
n_passed = (student_data.passed == 'yes').sum()

# Calculate failing students
n_failed = (student_data.passed == 'no').sum()

assert n_passed + n_failed == n_students, 'There are invalid labels on the dataset'

# Calculate graduation rate
grad_rate = n_passed / n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Samples per feature ratio: {}".format(n_students/n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

print("\nNumerical features:")
display(student_data.describe(include=[int, float]))

In [ ]:
plt.figure(figsize=(8,8))
plt.hist(student_data.absences, bins=20)
plt.axhline(y=student_data.absences.mean(), c='red')
plt.grid()
plt.show()

Looking at the plot above, we can see that `absences` is very skewed. We can apply a log transformation to make it behave well when fed to the model.

In [ ]:
plt.figure(figsize=(8,8))
plt.hist(np.log(student_data.absences + 1))
plt.grid()
plt.show()

## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [ ]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_raw = student_data.loc[:, feature_cols]
y_all = student_data.loc[:, target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
display(X_raw.head())

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

#### Extra: Feature Scaling
Some classifiers are sensitive about the scale of the features, like k-NN, SVM and regularized linear classifiers. Scaling features to similar ranges also provides more numerical stability, which is good for convergence of optimization algorithms.
Also, we have seen that `absences` is very skewed, so a log transform is applied to make it more normal.

In [ ]:
def preprocess_features(X, unskew_absences, scale_numeric):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    def is_binary(series):
        return set(series) == {'yes', 'no'}
    
    def minmax_scale(values):
        min_value = values.min()
        values_range = values.max() - min_value
        return (values - min_value)/values_range
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        if unskew_absences and col == 'absences':
            col_data = col_data.apply(lambda x: np.log(x + 1))
            
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            if is_binary(col_data):
                col_data = col_data.replace(['yes', 'no'], [1, 0])
                assert col_data.dtype == int
            else:  # If data type is categorical, convert to dummy variables
                # Example: 'school' => 'school_GP' and 'school_MS'
                col_data = pd.get_dummies(col_data, prefix = col)
        elif col_data.dtype in [int, float] and scale_numeric:
            print('Scaling feature ' + col)
            col_data = minmax_scale(col_data)
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_raw, False, False)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
display(X_all.head(1))

In [ ]:
def plot_tsne(X, y=None, perplexity=30.0):
    from sklearn.manifold.t_sne import TSNE
    import scipy.sparse as sp

    print('Embedding data into 2D, takes some time...')
    if sp.issparse(X):
        X = X.todense()
    Xt = TSNE(perplexity=perplexity).fit_transform(X, y)

    if y is None:
        plt.plot(list(Xt[:, 0]), list(Xt[:, 1]), marker='o', linestyle='')
    else:
        df = pd.DataFrame({'x': Xt[:, 0], 'y': Xt[:, 1], 'label': y})
        groups = df.groupby('label')

        fig, ax = plt.subplots()
        for name, group in groups:
            ax.plot(list(group.x), list(group.y), marker='o', linestyle='', label=name)

        ax.legend()

    plt.title('2D visualization of data')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

# Uncomment below to see 2D plot of data
plot_tsne(X_all, y_all, 50)

### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [ ]:
from sklearn.cross_validation import train_test_split

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = len(X_all) - num_train

# Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

The idea in choosing the model is to start with simple ones that might perform somewhat well on the data and move to more complicated ones (Occam's Razor). Also, diversity is important to make sure we are looking at the problem from different angles.

#### K-Nearest Neighbors
K-Nearest neighbors have been used in [behavioral targeting](https://en.wikipedia.org/wiki/Nearest_neighbor_search#Applications) to predict, for example, which ad is most likely to be clicked by a customer, based on previous activity of himself and others.
Being non-parametric, K-Nearest Neighbors can approximate arbitrarily complex functions (given enough data). It can deal with multiclass problems without needing any adjustment and can incorporate special domain knowledge by using a distance function tailored to the problem.
K-NN is a lazy learner, which means that almost no effort is done until we are prompted to estimate something. This is advantageous in settings where training the model is more common than querying it, but disadvantageous in the opposite scenario. Also, K-NN does not work well in high-dimensional problems due to the Curse of Dimensionality, although we can mitigate this using by dimensionality reduction techniques such as PCA.

I selected K-NN because the low-dimension embedding of the data suggested that there were some clusters for students that did not pass. (And also because I was curious about the structure of the data)


#### Decision Trees
Decision Trees are useful for [credit-scoring](http://www.knowledgetaiwan.org/ojs/index.php/ijbi/article/download/77/35), among [other applications](http://www.cbcb.umd.edu/~salzberg/docs/murthy_thesis/survey/node32.html), and are known for the ease of training on large datasets, capability of handling both numerical and categorical data, extensibility for multiclass problems, dealing with nonlinear decision boundaries and interpretability.
The problem with decision trees is that they too much attention to the data and tend to produce very complicated decision boundaries (overfitting), specially when classes are not axis-aligned. Prunning may be employed, but it may become difficult for the prunned tree to explain the training data effectively.
Decision Trees are a good starting point for the problem as they offer some insight into the data, are quick to train and, after hyper-parameter selection, can become good baseline models.

This model is a good candidate for the problem because we don't have to worry about having both numeric and categorical features or even feature scaling and both training and predicting should be fast. Also, we can inspect the model to see if it matches our understanding of the problem.


#### Random Forest
Random Forest have received a lot of attention in the field of [computer vision](http://pages.iai.uni-bonn.de/frintrop_simone//BVW13/BVW-gall.pdf) to predict, for example, handwritten digits or facial features in a face photograph.
Random Forests are very versatile, performing well on a variety of problems and offer a good starting point when not much is known about the data. They are able achieve balance between bias and variance by using bagging and attribute bagging for each tree that is built (which reduces correlation between trees).
Another key aspect of Random Forests is their capability of dealing with unimportant features, which turns to be a problem with Decision Trees (see above). Some even use them solely for [feature selection](http://scikit-learn.org/stable/modules/feature_selection.html#feature-selection-as-part-of-a-pipeline)!
Unfortunately, they are not as interpretable as decision trees: it is possible to see what features the model consider important, but that is all. Also, Random Forest will struggle with classes that are not axis-aligned (as Decision Trees do) or when the data is sparse (because of attribute bagging).

This model is a good candidate for the problem reasons similar to those of Decision Trees (interpretability to a lesser degree) with the advantages of increased predictive capacity and built-in feature selection mechanism.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

### To the reviewer - Please read
The `f1_score` function below was using `yes` as the positive label, which, to my understanding, does not match the goal of the assignment. On the **Project Overview** it was stated that
> Graduation rates are often the criteria of choice, and educators seek new ways to predict the success and failure of students early enough to stage effective interventions.

While the majority of students pass, we have to focus our attention on those who are likely to fail. Using `yes` as the positive label, a baseline model that always predicts `yes` will get a F<sub>1</sub> score of 0.7975 on the test set but won't provide any real value to our business.

To match the problem, an adjustment was made to `pos_label`, changing its value from `yes` to `no`.

In [ ]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in (seconds): {:.4f}".format(end - start)

    
def predict_labels(clf, features, target, source):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions for {} set in (seconds) {:.4f}".format(source, end - start)
    return f1_score(target.values, y_pred, pos_label='no')  # To reviewer: previously was YES. See notes above


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}".format(predict_labels(clf, X_train, y_train, 'train'))
    print "F1 score for test set: {:.4f}".format(predict_labels(clf, X_test, y_test, 'test'))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [ ]:
# Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier

# Initialize the three models
clf_A = KNeighborsClassifier()
clf_B = DecisionTreeClassifier(random_state=777)
clf_C = RandomForestClassifier(random_state=777)
clf_D = DummyClassifier(strategy='constant', constant='no')


# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

X_train_sized = [X_train_100, X_train_200, X_train_300]
y_train_sized = [y_train_100, y_train_200, y_train_300]

# Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C, clf_D]:
    for X_train_n, y_train_n in zip(X_train_sized, y_train_sized):
        train_predict(clf, X_train_n, y_train_n, X_test, y_test)
        print('\n')
    print('====')

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - K-Nearest Neighbors **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0011                  | 0.0018                 | 0.6061           | 0.2692          |
| 200               |        0.0014           | 0.0044                 | 0.6400           | 0.2979          |
| 300               | 0.0016                  | 0.0029                 | 0.6581           |    0.3673       |

** Classifer 2 - Decision Tree Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0013                  | 0.0004                 | 1.0000           | 0.4054          |
| 200               |     0.0021              | 0.0005                 | 1.0000           | 0.4138          |
| 300               | 0.0027                  | 0.0006                 | 1.0000           |     0.3385      |

** Classifer 3 - Random Forest Classifier **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0287                  | 0.0015                 | 0.9859           | 0.3774          |
| 200               | 0.0301                  | 0.0017                 | 0.9756           | 0.3600          |
| 300               | 0.0306                  | 0.0020                 | 0.9895           | 0.5902          |

** Naive Predictor **

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 300               | -                  | -                 | 0.4810           | 0.5385          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
Before we dive into the analysis, consider this: a model will be trained once in a while, but, after trained, will be used to make thousands (even more) predictions. Prediction time dominates the cost and will be the only one considered when choosing the model.

To estimate costs, we first need to estimate usage. According to [this](http://www.irrodl.org/index.php/irrodl/article/view/1651/2774), the average MOOC enroll around **43,000** students and has a typical length is 8-10 weeks. If we run our predictions on every enrolled student on a weekly basis, we will make a total of **344,000 predictions** for that course.

The first one to be ruled out is K-NN:
- Prediction time is the biggest of chosen models
- Score is much lower than Random Forest on both training and testing sets. The important metric is, of course, the testing score, but training score gives us an upper boundary to the testing score we can achieve.

To decide between Decision Trees and Random Forest, we should first analyze the costs.
- Decision Tree is faster than Random Forest (3-4x times faster when predicting). it would take about **0.7 seconds** (344000 * 0.0006 / 300) to make all predictions. Translating that time to costs, on a `c4.xlarge` EC2 instance, the cost of running this predictions would be only a fraction of a cent. So, both the costs of Decision Trees and Random Forests are negligible
- When compared to Random Forest, the F<sub>1</sub> score decreased by 0.25165 units. which is a lot! This means failing to take action on students that will likely abandon the course or unnecessarily tending to "good" students.

This leaves us with `Random Forest`, our winner. We make a conscious choice of trading predicting speed and interpretability of decision trees to achieve better predictions with RF. Also, the only one that fared better than the Naive Predictor.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [ ]:
# TODO: Import 'GridSearchCV' and 'make_scorer'

# TODO: Create the parameters list you wish to tune
parameters = None

# TODO: Initialize the classifier
clf = None

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = None

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = None

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = None

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.